# Imports

In [1]:
# Import modules
import logging
import threading
import time
import xlwings as xw

# Constants

In [2]:
# Excel font constants
HEADINGS_FONT = "Lato"
TABLE_FONT = "Source Sans Pro"
LARGE_FONT = 18
SMALL_FONT = 11

# Excel text alignment constants
XL_H_ALIGN_CENTER = -4108  # xlHAlignCenter
XL_V_ALIGN_CENTER = -4108  # xlVAlignCenter
XL_V_ALIGN_TOP = -4160  # xlVAlignTop



# Excel colour constants
MAPEI_GREY = 15132135  # RGB (231, 229, 230)
FULL_WHITE = 16777215  # RGB(255, 255, 255)

# Functions

In [3]:
def start_log():
    """
    Initializes the logging configuration for the application.
    """
    logging.basicConfig(
        filename='App.log',
        filemode='w',
        format='%(message)s',
        level=logging.DEBUG,
)


def end_log():
    """
    Closes the logging configuration for the application.
    """
    # Fetch the root logger
    logger = logging.getLogger()
    
    # Close all handlers of the logger
    for handler in logger.handlers[:]:
        handler.close()
        logger.removeHandler(handler)

# Classes

In [4]:
# Test Connection
class TestConnection:
    """
    Checks a connection can be established with Excel.
    """
    def __init__(self):
        """
        Initializes a connection to Excel, ensuring it's open and not in data entry mode.
        """
        # Establish an Excel connection
        self.xlApp = xw.apps.active
        if self.xlApp is None:
            raise Exception("Excel must be open to continue!")
        
        # Test if the workbook is accessible (not in data entry mode)
        self.workbook_connection_tested = False
        t1 = threading.Thread(target=self._test_connection)
        t2 = threading.Thread(target=self._connection_timer)
        t1.start(), t2.start()
        t1.join(), t2.join()
        self.wb = xw.books.active

    def _test_connection(self):
        """
        A thread to try and connect to Excel.
        """
        wb = xw.books.active
        wb = None
        self.workbook_connection_tested = True
        
    def _connection_timer(self):
        """
        A timer thread to identify if Excel is in data entry mode.
        """
        time.sleep(0.1)
        if not self.workbook_connection_tested:
            print("Excel is in data entry mode! Select Excel and press 'esc' key to continue.")  


# Import Data
class ImportData:
    def __init__(self, xlApp, wb, target_cells=("B3", "D3", "F3", "H3", "J3")):
        """
        Initializes the ImportData class.
        
        Parameters:
            xlApp (object): The Excel application object.
            wb (object): The Excel workbook
            target_cells (tuple): The cells in Excel to start reading data from.
        """
        self.xlApp = xlApp
        self.wb = wb
        self.ws = self._connect_to_worksheet()  
        self.target_cells = target_cells
        self.search_terms = []
        
        # Logging for debugging
        logging.info(f"self.xlApp = {self.xlApp}")
        logging.info(f"self.wb = {self.wb.fullname}")
        logging.info(f"self.ws = {self.ws.name}")
        logging.info(f"self.target_cells = {(self.target_cells)}")
        logging.info(f"__init__ function completed")

    def _connect_to_worksheet(self):
        """
        Connects to the desired worksheet.
        """
        try:
            ws = self.wb.sheets["Search Terms"]
            logging.info(f"self.ws = {ws.name}")
            logging.info(f"_connect_to_worksheet function completed")  
            return ws  # Return the worksheet object
        except:
            raise Exception("A 'Search Terms' worksheet could not be found.")
            

    def get_from_excel(self):
        """
        Imports data from the Excel sheet into the `search_terms` attribute.
        """
        all_empty = True # Flag to check if all the target_cells columns are empty
        for cell in self.target_cells:
            data_range = self.ws.range(cell).current_region.value
            
            # It data exists, append it to the search terms
            if data_range is not None:
                data = data_range[1:]
                if data[0] is not None:
                    self.search_terms.append(data)
                    logging.info(f"data = {data}")
                    all_empty = False # At least one column has data
        
        # Raise exception if all columns are empty
        if all_empty: raise Exception("All search term columns are empty!")
        
        # Logging for debugging            
        logging.info(f"self.search_terms {len(self.search_terms)} data groups added ")
        logging.info(f"import_data.get_from_excel function completed") 
        
        return self.search_terms
    
    
# Search Strings    
class SearchStrings:
    def __init__(self, search_terms):
        """
        Initializes the SearchStrings class.

        Parameters:
        search_terms (list): List of search terms to be used in creating search strings.
        """
        self.search_terms = search_terms
        self.basic_strings = []
        self.database_strings = {}

        # Logging for debugging
        logging.info(f"self.search_terms = {self.search_terms}")
        logging.info(f"__init__ function completed") 

    def make_strings(self):
        """
        Convert the search terms into database search strings.
        Calls helper functions to build search strings specific to each database.
        """
        self._make_basic_strings()  
        self._cinhal()
        self._medline_and_embase()
        self._scopus()
        self._web_of_science()
        
        # Logging for debugging
        logging.info(f"self.database_strings.keys = {self.database_strings.keys()}")
        logging.info(f"search_strings.make_strings function completed")
        return self.database_strings

    def _make_basic_strings(self):
        """
        Build basic search strings from the raw search terms.
        These basic search strings serve as the building blocks for database-specific search strings.
        """
        # Loop over each term group in search_terms
        for terms in self.search_terms:
            if terms[0] is not None:
                search_str = " or ".join([f'"{term}"' for term in terms])
                self.basic_strings.append(search_str)
        
        logging.info(f"self.basic_strings = {self.basic_strings}")
        logging.info(f"_make_basic_strings function completed") 
             
    def _cinhal(self, search_type=["TI", "AB"]):
        """
        Constructs Cinhal-specific search strings.

        Parameters:
        search_type (list): List of search types to be used in creating Cinhal-specific search strings.

        Appends each constructed search string as a dictionary to the 'Cinhal' key in self.database_strings.
        """
        self.database_strings['Cinhal'] = []  
        
        # Loop over each search_type to build the complete search string
        for counter, st in enumerate(search_type):
            search_strings = [f"{st}({basic_string})" for basic_string in self.basic_strings]
            complete_search_string = f" AND ".join(search_strings)
            
            # Create a dictionary with all the needed information for this search string
            new_entry = {
                'ws_index': 'cinhal',
                'ws_heading':'Cinhal String',
                'search_type': st,
                'counter': counter,
                'search_string': complete_search_string
            }

            self.database_strings['Cinhal'].append(new_entry)

        # Logging for debugging
        logging.info(f"self.database_strings['Cinhal'] list length) = {len(self.database_strings['Cinhal'])}")
        logging.info(f"self.database_strings['Cinhal'][0].keys = {self.database_strings['Cinhal'][0].keys()}")
        logging.info(f"_cinhal function completed")    

    def _medline_and_embase(self, search_type=[".m_titl.", ".ab", ".ti,ab."]):
        """
        Constructs Medline and Embase-specific search strings.

        Parameters:
        search_type (list): List of search types to be used in creating Medline and Embase-specific search strings.

        Appends each constructed search string as a dictionary to the 'medline and embase' key in self.database_strings.
        """
        self.database_strings['medline and embase'] = []
        
        # Loop over each search_type to build the complete search string
        for counter, st in enumerate(search_type):
            search_strings = [f"{st}({basic_string})" for basic_string in self.basic_strings]
            complete_search_string = f" AND ".join(search_strings)
            
            # Create a dictionary with all the needed information for this search string
            new_entry = {
                'ws_index': 'medline and embase',
                'ws_heading': 'Medline and Embase String',
                'search_type': st,
                'counter': counter,
                'search_string': complete_search_string
            }

            self.database_strings['medline and embase'].append(new_entry)
            
        # Logging for debugging
        logging.info(f"self.database_strings['medline and embase'] list length) = {len(self.database_strings['medline and embase'])}")
        logging.info(f"self.database_strings['medline and embase'][0].keys = {self.database_strings['medline and embase'][0].keys()}")
        logging.info(f"_medline_and_embase function completed")

    def _scopus(self):
        """
        Constructs Scopus search strings nd appends them to the 'Scopus' key 
        in the self.database_strings dictionary.
        """
        self.database_strings['Scopus'] = []
        search_strings = [f"({basic_string})" for basic_string in self.basic_strings]
        complete_search_string = f" AND ".join(search_strings)
        complete_search_string = f"(({complete_search_string}))"
        
        # Create a dictionary with all the needed information for this search string
        new_entry = {
                'ws_index': 'scopus',
                'ws_heading':'Scopus String',
                'search_type': "",
                'counter': 0,
                'search_string': complete_search_string
            }
        
        self.database_strings['Scopus'].append(new_entry)
        
        # Logging for debugging
        logging.info(f"self.database_strings['Scopus'] list length) = {len(self.database_strings['Scopus'])}")
        logging.info(f"self.database_strings['Scopus'][0].keys = {self.database_strings['Scopus'][0].keys()}")
        logging.info(f"_scopus function completed")

    def _web_of_science(self):
        """
        Constructs Web of Science-specific search strings and appends them to the 'Web of Science' key 
        in the self.database_strings dictionary.
        """
        self.database_strings['Web of Science'] = []
        search_strings = [f"({basic_string})" for basic_string in self.basic_strings]
        complete_search_string = f" AND ".join(search_strings)
        complete_search_string = f"(({complete_search_string}))"
        
        # Create a dictionary with all the needed information for this search string
        new_entry = {
                'ws_index': 'web of science',
                'ws_heading':'Web of Science String',
                'search_type': "",
                'counter': 0,
                'search_string': complete_search_string
            }
        self.database_strings['Web of Science'].append(new_entry)

        # Logging for debugging
        logging.info(f"self.database_strings['Web of Science'] list length) = {len(self.database_strings['Web of Science'])}")
        logging.info(f"self.database_strings['Web of Science'][0].keys = {self.database_strings['Web of Science'][0].keys()}")
        logging.info(f"_web_of_science function completed")
      
      
# Export Data
class ExportData:
    def __init__(self, xlApp, wb, database_strings):
        """
        Initializes the ExportData class.

        Parameters:
            xlApp (object): The Excel application object.
            wb (object): The Excel workbook.
            database_strings (dict): The database-specific search strings.
        """
        self.xlApp = xlApp
        self.wb = wb
        self.database_strings = database_strings

    def send_to_excel(self):
        """
        Loops through all database keys and entries to send data to Excel.
        """
        for db_key in self.database_strings.keys():
            logging.info(f"Exporting {db_key} strings")
            for entry in self.database_strings[db_key]:
                self._export_string(entry['ws_heading'], entry['ws_index'], entry['search_string'], entry['counter'])

    def _export_string(self, ws_title, ws_index, complete_search_string, counter):
        """
        Exports a single entry to the Excel workbook.

        Parameters:
            ws_title (str): The title for the worksheet.
            ws_index (str): The index name for the worksheet.
            complete_search_string (str): The complete search string for this entry.
            counter (int): The row index for this entry.
        """
        # Try to select the sheet, if it doesn't exist, create it
        try:
            self.wb.sheets[ws_index].select()
        except:
            self.wb.sheets.add(name=ws_index, after=self.wb.sheets.count)
        finally:
            self.ws = self.wb.sheets.active
        
        # If this is the first entry, set up the worksheet
        if counter == 0:
            # Format the first row and column
            self.ws.range("A:A").column_width = 0.5
            self.ws.range("1:1").row_height = 5

            # Format the title bar
            self.ws.range("B2:B4").api.Interior.Color = MAPEI_GREY
            self.ws.range("2:4").row_height = 21

            # Format the body text
            self.ws.range("B:B").font.name = TABLE_FONT
            self.ws.range("B:B").font.size = SMALL_FONT
            self.ws.range("B:B").wrap_text = True
            self.ws.range("B:B").api.VerticalAlignment = XL_V_ALIGN_TOP
            self.ws.range("B:B").column_width = 150
            
            # Format the heading text
            self.ws.range("B3").font.name = HEADINGS_FONT
            self.ws.range("B3").font.size = LARGE_FONT
            self.ws.range("B3").font.bold = True
            self.ws.range("B3").font.color = FULL_WHITE
            
            # Add the title string
            self.ws.range("B3").value = ws_title

        # Add the complete search string
        row_idx = str(counter + 5)
        self.ws.range(f"{row_idx}:{row_idx}").row_height = 120
        self.ws.range(f"B{row_idx}").value = complete_search_string

        # Set the final cell selection
        self.ws.range("B7").select()
        logging.info(f"export_data._export_string function complete")

# Main 

In [5]:
def make_search_strings():
    """
    Orchestrator function that controls the overall execution of the application.
    
    Steps:
    1. Tests the connection to an open Excel.
    2. Imports search terms from a "search_terms" Excel worksheet.
    3. Processes the imported search terms to create database-specific search strings.
    4. Exports the search strings to Excel.
    """

    # Run a test to check connection with Excel application. 
    test_connection = TestConnection()
    xlApp = test_connection.xlApp
    wb = test_connection.wb

    # Import the search terms from Excel.
    logging.info(f"Running ImportData Class  -------------")
    import_data = ImportData(xlApp, wb)
    search_terms = import_data.get_from_excel()
    logging.info(f"-----------------------------------------\n\n")
    
    # Create the database search strings.
    logging.info(f"Running SearchStrings Class  ----------")
    search_strings = SearchStrings(search_terms)
    database_strings = search_strings.make_strings()
    logging.info(f"-----------------------------------------\n\n")

    # Export the strings to Excel.
    logging.info(f"Running ExportData Class  -------------")
    export_data = ExportData(xlApp, wb, database_strings)
    export_data.send_to_excel()
    logging.info(f"-----------------------------------------\n\n")

    # Confirmation statement
    print("Task Complete...")


def main():
    # Enable or disable logging 
    start_log()

    # Call make_search_strings function (cinhal, embase/medline, scopus, web of science)
    make_search_strings()

    # End logging and close file
    end_log()

if __name__ == "__main__":
    main()    

Task Complete...
